### Imports

In [9]:
import math, requests, json

### Constants

In [2]:
# Constants
TWOMBA_ENABLED = True 
TAKER_FEE_CONSTANT = 0.07
CREATORS_EARN_WHOLE_FEE_UP_TO = 1000

# Flat fees
FLAT_TRADE_FEE = 0.1
FLAT_COMMENT_FEE = 1

DPM_PLATFORM_FEE = 0.0
DPM_CREATOR_FEE = 0.0
DPM_FEES = DPM_PLATFORM_FEE + DPM_CREATOR_FEE

# Math
EPSILON = 0.00000001

def floatingEqual(a: float, b: float, epsilon = EPSILON):
    return abs(a - b) < epsilon

### Calculate Fees

In [3]:
class Fees:
    def __init__(self, creatorFee=0, platformFee=0, liquidityFee=0):
        self.creatorFee = creatorFee
        self.platformFee = platformFee
        self.liquidityFee = liquidityFee
    
    def __repr__(self):
        return f"Fees(creatorFee={self.creatorFee}, platformFee={self.platformFee}, liquidityFee={self.liquidityFee})"
    
    def add(self, other):
        return Fees(
            self.creatorFee + other.creatorFee,
            self.platformFee + other.platformFee,
            self.liquidityFee + other.liquidityFee
        )

def addObjects(obj1, obj2):
    return obj1.add(obj2)

def getTakerFee(shares: float, prob: float) -> float:
    return TAKER_FEE_CONSTANT * prob * (1 - prob) * shares

def getFeesSplit(totalFees, previouslyCollectedFees) -> Fees:
    if TWOMBA_ENABLED:
        return Fees(0, totalFees, 0)
    
    before1k = max(0, CREATORS_EARN_WHOLE_FEE_UP_TO - previouslyCollectedFees.creatorFee)
    allToCreatorAmount = min(totalFees, before1k)
    splitWithCreatorAmount = totalFees - allToCreatorAmount
    
    return Fees(
        creatorFee=allToCreatorAmount + splitWithCreatorAmount * 0.5,
        platformFee=splitWithCreatorAmount * 0.5,
        liquidityFee=0
    )

def getFeeTotal(fees) -> float:
    return fees.creatorFee + fees.platformFee + fees.liquidityFee

def sumAllFees(fees_list) -> float:
    total_fees = Fees()
    for fee in fees_list:
        total_fees = addObjects(total_fees, fee)
    return getFeeTotal(total_fees)


### Calculate CPMM

In [8]:
class CpmmState:
    def __init__(self, pool: dict[str, float], p: float, collectedFees: Fees):
        self.pool = pool
        self.p = p
        self.collectedFees = collectedFees
        
def getCpmmProbability(pool: dict[str, float], p: float) -> float:
    YES, NO = pool['YES'], pool['NO']
    return (p * NO) / ((1 - p) * YES + p * NO)

def getCpmmProbabilityAfterBetBeforeFees(state: CpmmState, outcome: str, bet: float) -> float:
    pool, p = state.pool, state.p
    shares = calculateCpmmShares(pool, p, bet, outcome)
    YES, NO = pool['YES'], pool['NO']
    
    if outcome == "YES":
        newY, newN = YES - shares + bet, NO + bet
    else:
        newY, newN = YES + bet, NO - shares + bet
    
    return getCpmmProbability({'YES': newY, 'NO': newN}, p)

def getCpmmOutcomeProbabilityAfterBet(state: CpmmState, outcome: str, bet: float) -> float:
    newPool = calculateCpmmPurchase(state, bet, outcome)[1]
    p = getCpmmProbability(newPool, state.p)
    if outcome == 'NO':
        return 1 - p
    else:
        return p

# before liquidty fee
def calculateCpmmShares(pool: dict[str, float], p: float, betAmount: float, betChoice: str) -> float:
    if betAmount == 0:
        return 0
    
    YES, NO = pool['YES'], pool['NO']
    k = (YES ** p) * (NO ** (1 - p))
    
    # https://www.wolframalpha.com/input?i=%28y%2Bb-s%29%5E%28p%29*%28n%2Bb%29%5E%281-p%29+%3D+k%2C+solve+s
    if betChoice == 'YES':
        return YES + betAmount - (k * (betAmount + NO) ** (p - 1)) ** (1 / p)
    elif betChoice == 'NO':
        return NO + betAmount - (k * (betAmount + YES) ** -p) ** (1 / (1 - p))

def getCpmmFees(state: CpmmState, betAmount: float, outcome: str) -> tuple:
    fee = 0
    for _ in range(10):
        betAmountAfterFee = betAmount - fee
        shares = calculateCpmmShares(state.pool, state.p, betAmountAfterFee, outcome)
        averageProb = betAmountAfterFee / shares
        fee = getTakerFee(shares, averageProb)
    
    totalFees = 0 if betAmount == 0 else fee
    fees = getFeesSplit(totalFees, state.collectedFees)
    remainingBet = betAmount - totalFees
    
    return remainingBet, totalFees, fees

def calculateCpmmSharesAfterFee(state: CpmmState, bet: float, outcome: str) -> float:
    pool, p = state.pool, state.p
    remainingBet = getCpmmFees(state, bet, outcome)[0]
    
    return calculateCpmmShares(pool, p, remainingBet, outcome)

def calculateCpmmPurchase(state: CpmmState, bet: float, outcome: str, freeFees:bool=False) -> tuple:
    pool, p = state.pool, state.p
    remainingBet, fees = (bet, Fees()) if freeFees else (getCpmmFees(state, bet, outcome)[0], getCpmmFees(state, bet, outcome)[2])
    
    shares = calculateCpmmShares(pool, p, remainingBet, outcome)
    YES, NO = pool['YES'], pool['NO']
    
    fee = fees.liquidityFee 
    
    if outcome == 'YES':
        newY, newN = YES - shares + remainingBet + fee, NO + remainingBet + fee
    else:
        newY, newN = YES + remainingBet + fee, NO - shares + remainingBet + fee    
    
    postBetPool = {'YES': newY, 'NO': newN}

    newPool, newP = addCpmmLiquidity(postBetPool, p, fee)[0], addCpmmLiquidity(postBetPool, p, fee)[2]
    
    return shares, newPool, newP, fees

def calculateCpmmAmountToProb(state: CpmmState, prob: float, outcome: str) -> float:
    if prob <= 0 or prob >= 1 or math.isnan(prob):
        return float('inf')
    if outcome == 'NO':
        prob = 1 - prob
    
    pool, p = state.pool, state.p
    YES, NO = pool['YES'], pool['NO']
    k = YES ** p * NO ** (1 - p)
    # https://www.wolframalpha.com/input?i=-1+%2B+t+-+((-1+%2B+p)+t+(k%2F(n+%2B+b))^(1%2Fp))%2Fp+solve+b
    if outcome == 'YES':
        ((p * (prob - 1)) / ((p - 1) * prob)) ** -p * (k - NO * ((p * (prob - 1)) / ((p - 1) * prob)) ** p)
    else:
        (((1 - p) * (prob - 1)) / (-p * prob)) ** (p - 1) * (k - YES * (((1 - p) * (prob - 1)) / (-p * prob)) ** (1 - p))

def calculateCpmmAmountToProbIncludingFees(state: CpmmState, prob: float, outcome: str) -> float:
    amount = calculateCpmmAmountToProb(state, prob, outcome)
    shares = calculateCpmmShares(state.pool, state.p, amount, outcome)
    averageProb = amount / shares
    fees = getTakerFee(shares, averageProb)
    return amount + fees

def calculateCpmmAmountToBuySharesFixedP(state: CpmmState, shares: float, outcome: str) -> float:
    if not floatingEqual(state.p, 0.5):
        raise ValueError(f"calculateAmountToBuySharesFixedP only works for p = 0.5, got {state.p}")
    
    YES, NO = state.pool['YES'], state.pool['NO']
    # https://www.wolframalpha.com/input?i=%28y%2Bb-s%29%5E0.5+*+%28n%2Bb%29%5E0.5+%3D+y+%5E+0.5+*+n+%5E+0.5%2C+solve+b
    if outcome == 'YES':
        return (shares - YES - NO + math.sqrt(4 * NO * shares + (YES + NO - shares) ** 2)) / 2
    else:
        return (shares - YES - NO + math.sqrt(4 * YES * shares + (YES + NO - shares) ** 2)) / 2
    
# Faster version assuming p = 0.5
def calculateAmountToBuySharesFixedP(state: CpmmState, shares: float, outcome: str, unfilledBets):
    pass

def getCpmmLiquidity(pool: dict[str, float], p: float) -> float:
    YES, NO = pool['YES'], pool['NO']
    return (YES ** p) * (NO ** (1 - p))

def addCpmmLiquidity(pool: dict[str, float], p: float, amount: float) -> tuple:
    prob = getCpmmProbability(pool, p)
    
    # https://www.wolframalpha.com/input?i=p%28n%2Bb%29%2F%28%281-p%29%28y%2Bb%29%2Bp%28n%2Bb%29%29%3Dq%2C+solve+p
    YES, NO = pool['YES'], pool['NO']
    numerator = prob * (amount + YES)
    denominator = amount - NO * (prob - 1) + prob * YES
    newP = numerator / denominator
    newPool = {'YES': YES + amount, 'NO': NO + amount}
    
    oldLiquidity = getCpmmLiquidity(pool, newP)
    newLiquidity = getCpmmLiquidity(newPool, newP)
    liquidity = newLiquidity - oldLiquidity
    
    return newPool, liquidity, newP


### Calculate Multi CPMM

In [ ]:
def getK(pool: dict[str, float]) -> float:
    values = pool.values()
    return sum(map(math.log, values))

def getLiquidity(pool: dict[str, float]):
    return math.exp(getK(pool) / len(pool))

### Test Cases

- **Manifold API**: https://docs.manifold.markets/api
- **GET request URL**: https://api.manifold.markets/v0
- **GET by market slug**: https://manifold.markets/api/v0/slug/
- **GET by market ID**: https://api.manifold.markets/v0/market/
- **GET bets**: https://api.manifold.markets/v0/bets/
- **p**: This is the probability around which the market liquidity is "centered'. Probability constant in $y^{p}n^{1-p} = k$

In [24]:
# Will Ken Klippenstein's account be unbanned from Twitter / X by the end of October?

id = "04eacvnevn"
slug = "will-ken-klippensteins-account-be-u-fdh0tjsfga"

rooturl = "https://api.manifold.markets/v0/market/"
data = requests.get(rooturl + id).json()

n = data['pool']['NO']
y = data['pool']['YES']
prob = data['probability']
p = data['p']
liquidity = data['totalLiquidity']
volume = data['volume']

print(f'{n = }\n{y = }\n{prob = }\n{p = }\n{liquidity = }\n{volume = }')

k = (y**p) * (n**(1 - p))
print(f'{k = }') 

betsURL = "https://api.manifold.markets/v0/bets?contractId="
betsData = requests.get(betsURL+id).json()

creatorFee = 0
platformFee = 0
liquidityFee = 0
trades = []
for i in betsData:
    if i['isCancelled'] != None:
        trades.append(i)
    creatorFee += i['fees']['creatorFee']
    platformFee += i['fees']['platformFee']
    liquidityFee += i['fees']['liquidityFee']

print(f'Number of Bets = {len(trades)}')

print()
print(f'{creatorFee = }\n{platformFee = }\n{liquidityFee = }')

collectedFees = Fees(creatorFee, platformFee, liquidityFee)
pool = {'YES': y, 'NO': n}
state = CpmmState(pool, p, collectedFees)

# New Bet
bet = 100
outcome = 'NO'
newProb = getCpmmOutcomeProbabilityAfterBet(state, outcome, bet)
newProbBeforeFees = getCpmmProbabilityAfterBetBeforeFees(state, outcome, bet)
totalFees = getCpmmFees(state, bet, outcome)[1]
maxPayout = calculateCpmmSharesAfterFee(state, bet, outcome)
print()
print(f"{newProb = }\n{newProbBeforeFees = }\n{totalFees = }\n{maxPayout = }")

n = 1045.625089882319
y = 968.3027243909308
prob = 0.7096077023419292
p = 0.6935256453764421
liquidity = 1000
volume = 167.47012668720356
k = 991.3719394776825
Number of Bets = 7

creatorFee = 0
platformFee = 3.8974977545342067
liquidityFee = 0
13.827914927896451

newProb = 0.3574038203420311
newProbBeforeFees = 0.6394077993245363
totalFees = 4.5267610836093
maxPayout = 295.89590116260547


In [6]:
# Will UniCredit successfully acquire a majority stake in Commerzbank by 2025?

id = "spqjzzszu4"
slug = "will-unicredit-successfully-acquire"

rooturl = "https://api.manifold.markets/v0/market/"
data = requests.get(rooturl + id).json()

n = data['pool']['NO']
y = data['pool']['YES']
prob = data['probability']
p = data['p']
liquidity = data['totalLiquidity']
volume = data['volume']

print(f'{n = }\n{y = }\n{prob = }\n{p = }\n{liquidity = }\n{volume = }')

k = (y**p) * (n**(1 - p))
print(f'{k = }')

betsURL = "https://api.manifold.markets/v0/bets?contractId="
betsData = requests.get(betsURL+id).json()

creatorFee = 0
platformFee = 0
liquidityFee = 0
trades = []
for i in betsData:
    if i['isCancelled'] != None:
        trades.append(i)
    creatorFee += i['fees']['creatorFee']
    platformFee += i['fees']['platformFee']
    liquidityFee += i['fees']['liquidityFee']

print(f'Number of Bets = {len(trades)}')

print()
print(f'{creatorFee = }\n{platformFee = }\n{liquidityFee = }')

collectedFees = Fees(creatorFee, platformFee, liquidityFee)
state = CpmmState({'YES': y, 'NO': n}, p, collectedFees)

# New Bet
bet = 100
outcome = 'NO'
newProb = getCpmmOutcomeProbabilityAfterBet(state, outcome, bet)
newProbBeforeFees = getCpmmProbabilityAfterBetBeforeFees(state, outcome, bet)
totalFees = getCpmmFees(state, bet, outcome)[1]
maxPayout = calculateCpmmSharesAfterFee(state, bet, outcome)
print()
print(f"{newProb = }\n{newProbBeforeFees = }\n{totalFees = }\n{maxPayout = }")

n = 1098.9304626378225
y = 909.975684539353
prob = 0.5470292697257112
p = 0.49999999999999994
liquidity = 1000
volume = 200
k = 999.9999999999999
Number of Bets = 3

creatorFee = 0
platformFee = 6.603704976775171
liquidityFee = 0

newProb = 0.5032149661911548
newProbBeforeFees = 0.49503703507523134
totalFees = 3.5249465770204016
maxPayout = 201.81490871065216


In [25]:
# Will the total number of Giving What We Can pledges made in 2024 be higher than in 2022?

id = "Y3lXD7s2CWFoRwHcnlm4"
slug = "will-the-total-number-of-giving-wha-cc4cca3a58ce"

rooturl = "https://api.manifold.markets/v0/market/"
data = requests.get(rooturl + id).json()

n = data['pool']['NO']
y = data['pool']['YES']
prob = data['probability']
p = data['p']
liquidity = data['totalLiquidity']
volume = data['volume']

print(f'{n = }\n{y = }\n{prob = }\n{p = }\n{liquidity = }\n{volume = }')

k = (y**p) * (n**(1 - p))
print(f'{k = }') 

betsURL = "https://api.manifold.markets/v0/bets?contractId="
betsData = requests.get(betsURL+id).json()

creatorFee = 0
platformFee = 0
liquidityFee = 0
trades = []
for i in betsData:
    if i['isCancelled'] != None:
        trades.append(i)
    creatorFee += i['fees']['creatorFee']
    platformFee += i['fees']['platformFee']
    liquidityFee += i['fees']['liquidityFee']

print(f'Number of Bets = {len(trades)}')

print()
print(f'{creatorFee = }\n{platformFee = }\n{liquidityFee = }')

collectedFees = Fees(creatorFee, platformFee, liquidityFee)
state = CpmmState({'YES': y, 'NO': n}, p, collectedFees)

# New Bet
bet = 100
outcome = 'YES'
newProb = getCpmmOutcomeProbabilityAfterBet(state, outcome, bet)
totalFees = getCpmmFees(state, bet, outcome)[1]
maxPayout = calculateCpmmSharesAfterFee(state, bet, outcome)
print()
print(f"{newProb = }\n{totalFees = }\n{maxPayout = }")

n = 581.7660652638804
y = 2962.1948186564964
prob = 0.07337731030907553
p = 0.2873448976738495
liquidity = 1000
volume = 4589.108025636279
k = 928.6723240171632
Number of Bets = 57

creatorFee = 5.772602170741537
platformFee = 16.271986404137195
liquidityFee = 0

newProb = 0.1176854020389118
totalFees = 5.971271104375298
maxPayout = 1013.3579146920897


In [26]:
# Will Vladimir Putin attend the G20 in Brazil in 2024?

id = "fN12OYGS30DGubdZuKEC"
slug = "will-vladimir-putin-attend-the-g20-de36a6365070"

rooturl = "https://api.manifold.markets/v0/market/"
data = requests.get(rooturl + id).json()

n = data['pool']['NO']
y = data['pool']['YES']
prob = data['probability']
p = data['p']
liquidity = data['totalLiquidity']
volume = data['volume']

print(f'{n = }\n{y = }\n{prob = }\n{p = }\n{liquidity = }\n{volume = }')

k = (y**p) * (n**(1 - p))
print(f'{k = }')

betsURL = "https://api.manifold.markets/v0/bets?contractId="
betsData = requests.get(betsURL+id).json()

creatorFee = 0
platformFee = 0
liquidityFee = 0
trades = []
for i in betsData:
    if i['isCancelled'] != None:
        trades.append(i)
    creatorFee += i['fees']['creatorFee']
    platformFee += i['fees']['platformFee']
    liquidityFee += i['fees']['liquidityFee']

print(f'Number of Bets = {len(trades)}')

print()
print(f'{creatorFee = }\n{platformFee = }\n{liquidityFee = }')

collectedFees = Fees(creatorFee, platformFee, liquidityFee)
state = CpmmState({'YES': y, 'NO': n}, p, collectedFees)

# New Bet
bet = 100
outcome = 'NO'
newProb = getCpmmOutcomeProbabilityAfterBet(state, outcome, bet)
totalFees = getCpmmFees(state, bet, outcome)[1]
maxPayout = calculateCpmmSharesAfterFee(state, bet, outcome)
print()
print(f"{newProb = }\n{totalFees = }\n{maxPayout = }")

n = 845.488397300625
y = 1895.3144343055671
prob = 0.09377010146250721
p = 0.1882805892896765
liquidity = 1000
volume = 1898.9589602278047
k = 984.2691189518603
Number of Bets = 48

creatorFee = 6.122965415212506
platformFee = 3.627075266532548
liquidityFee = 0

newProb = 0.9114443014600906
totalFees = 0.633803078299758
maxPayout = 109.32827980790546


In [22]:
# Get all my bets
import urllib.request
from prettytable import PrettyTable

url = "https://api.manifold.markets/v0/bets?username=Abdullah"
request = urllib.request.urlopen(url)

data = json.loads(request.read())

header =['id', 'amount', 'shares', 'outcome', 'probBefore', 'probAfter', ]

dict_keys(['id', 'fees', 'fills', 'isApi', 'amount', 'shares', 'userId', 'outcome', 'isFilled', 'probAfter', 'contractId', 'loanAmount', 'probBefore', 'visibility', 'createdTime', 'isCancelled', 'orderAmount', 'isRedemption', 'betId', 'updatedTime'])
